### Implementing FunkSVD - Solution

In this notebook we will take a look at writing our own function that performs FunkSVD, which will follow the steps you saw in the previous video.  If you find that you aren't ready to tackle this task on your own, feel free to skip to the following video where you can watch as I walk through the steps.

To test our algorithm, we will run it on the subset of the data you worked with earlier.  Run the cell below to get started.

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy import sparse
import svd_tests as t
%matplotlib inline

# Read in the datasets
movies = pd.read_csv('movies_clean.csv')
reviews = pd.read_csv('reviews_clean.csv')

del movies['Unnamed: 0']
del reviews['Unnamed: 0']

# Create user-by-item matrix
user_items = reviews[['user_id', 'movie_id', 'rating', 'timestamp']]
user_by_movie = user_items.groupby(['user_id', 'movie_id'])['rating'].max().unstack()

# Create data subset
user_movie_subset = user_by_movie[[73486, 75314,  68646, 99685]].dropna(axis=0)
ratings_mat = np.matrix(user_movie_subset)
print(ratings_mat)

[[10. 10. 10. 10.]
 [10.  4.  9. 10.]
 [ 8.  9. 10.  5.]
 [ 9.  8. 10. 10.]
 [10.  5.  9.  9.]
 [ 6.  4. 10.  6.]
 [ 9.  8. 10.  9.]
 [10.  5.  9.  8.]
 [ 7.  8. 10.  8.]
 [ 9.  5.  9.  7.]
 [ 9.  8. 10.  8.]
 [ 9. 10. 10.  9.]
 [10.  9. 10.  8.]
 [ 5.  8.  5.  8.]
 [10.  8. 10. 10.]
 [ 9.  9. 10. 10.]
 [ 9.  8.  8.  8.]
 [10.  8.  1. 10.]
 [ 5.  6. 10. 10.]
 [ 8.  7. 10.  7.]]


`1.` You will use the **user_movie_subset** matrix to show that your FunkSVD algorithm will converge.  In the below cell, use the comments and document string to assist you as you complete writing your own function to complete FunkSVD.  You may also want to try to complete the funtion on your own without the assistance of comments.  You may feel free to remove and add to the function in any way that gets you a working solution! 

**Notice:** There isn't a sigma matrix in this version of matrix factorization.

In [2]:
def FunkSVD(ratings_mat, latent_features=4, learning_rate=0.0001, iters=100):
    '''
    This function performs matrix factorization using a basic form of FunkSVD with no regularization
    
    INPUT:
    ratings_mat - (numpy array) a matrix with users as rows, movies as columns, and ratings as values
    latent_features - (int) the number of latent features used
    learning_rate - (float) the learning rate 
    iters - (int) the number of iterations
    
    OUTPUT:
    user_mat - (numpy array) a user by latent feature matrix
    movie_mat - (numpy array) a latent feature by movie matrix
    '''
    
    # Set up useful values to be used through the rest of the function
    n_users = ratings_mat.shape[0]
    n_movies = ratings_mat.shape[1]
    num_ratings = np.count_nonzero(~np.isnan(ratings_mat))
    
    # initialize the user and movie matrices with random values
    user_mat = np.random.rand(n_users, latent_features)
    movie_mat = np.random.rand(latent_features, n_movies)
    
    # initialize sse at 0 for first iteration
    sse_accum = 0
    
    # header for running results
    print("Optimizaiton Statistics")
    print("Iterations | Mean Squared Error ")
    
    # for each iteration
    for iteration in range(iters):

        # update our sse
        old_sse = sse_accum
        sse_accum = 0
        
        # For each user-movie pair
        for i in range(n_users):
            for j in range(n_movies):
                
                # if the rating exists
                if ratings_mat[i, j] > 0:
                    
                    # compute the error as the actual minus the dot product of the user and movie latent features
                    diff = ratings_mat[i, j] - np.dot(user_mat[i, :], movie_mat[:, j])
                    
                    # Keep track of the sum of squared errors for the matrix
                    sse_accum += diff**2
                    
                    # update the values in each matrix in the direction of the gradient
                    for k in range(latent_features):
                        user_mat[i, k] += learning_rate * (2*diff*movie_mat[k, j])
                        movie_mat[k, j] += learning_rate * (2*diff*user_mat[i, k])

        # print results for iteration
        print("%d \t\t %f" % (iteration+1, sse_accum / num_ratings))
        
    return user_mat, movie_mat 

`2.` Try out your function on the **user_movie_subset** dataset.  First try 4 latent features, a learning rate of 0.005, and 10 iterations.  When you take the dot product of the resulting U and V matrices, how does the resulting **user_movie** matrix compare to the original subset of the data?

In [3]:
user_mat, movie_mat = FunkSVD(ratings_mat, latent_features=4, learning_rate=0.005, iters=10)

Optimizaiton Statistics
Iterations | Mean Squared Error 
1 		 38.560357
2 		 12.087675
3 		 3.615695
4 		 2.763314
5 		 2.690298
6 		 2.665838
7 		 2.645359
8 		 2.623532
9 		 2.599299
10 		 2.572270


In [4]:
print(np.dot(user_mat, movie_mat))
print(ratings_mat)

[[10.24870999  8.89115195 10.50013981 10.14061152]
 [ 9.14947038  7.47868166  9.17570435  8.38214419]
 [ 7.8262065   7.05882467  8.35368036  7.61081248]
 [ 9.18548655  8.61126941 10.02965349  9.60983774]
 [ 8.62777308  7.33956861  8.87817772  8.6113624 ]
 [ 6.73673339  5.92179788  7.12618798  7.02875686]
 [ 9.27103905  8.08854827  9.66499383  9.14731618]
 [ 8.28675229  7.21183608  8.70226712  8.04397151]
 [ 8.33858051  7.46901709  8.81092672  8.61890039]
 [ 8.09303909  6.62535313  8.01663882  7.52862735]
 [ 9.19791248  7.80827498  9.33907504  8.6295149 ]
 [ 9.39819173  8.3535673  10.2702307   9.5048489 ]
 [ 9.39904572  7.85582453  9.71132428  9.31770017]
 [ 6.21829407  5.96515511  6.85939085  6.85390489]
 [ 9.51359224  8.60196663 10.2052838   9.85114047]
 [ 9.55439044  8.69334284 10.25165652  9.73229183]
 [ 8.84954064  7.10542745  8.58235073  8.15526447]
 [ 7.0904136   6.12761508  7.23219657  7.31802981]
 [ 7.99649146  7.40551892  8.8172377   8.54682501]
 [ 8.23199611  7.01137902  8.37

**The predicted ratings from the dot product are already starting to look a lot like the original data values even after only 10 iterations.  You can see some extreme low values that are not captured well yet.  The 5 in the second to last row in the first column is predicted as an 8, and the 4 in the second row and second column is predicted to be a 7.  Clearly the model is not done learning, but things are looking good.**

`3.` Let's try out the function again on the **user_movie_subset** dataset.  This time we will again use 4 latent features and a learning rate of 0.005.  However, let's bump up the number of iterations to 250.  When you take the dot product of the resulting U and V matrices, how does the resulting **user_movie** matrix compare to the original subset of the data?  What do you notice about your error at the end of the 250 iterations?

In [5]:
user_mat, movie_mat = FunkSVD(ratings_mat, latent_features=4, learning_rate=0.005, iters=250)

Optimizaiton Statistics
Iterations | Mean Squared Error 
1 		 45.038240
2 		 15.974660
3 		 4.356726
4 		 2.855060
5 		 2.687044
6 		 2.640645
7 		 2.612543
8 		 2.584797
9 		 2.553328
10 		 2.516948
11 		 2.474967
12 		 2.426732
13 		 2.371597
14 		 2.308971
15 		 2.238389
16 		 2.159593
17 		 2.072613
18 		 1.977850
19 		 1.876130
20 		 1.768733
21 		 1.657355
22 		 1.544013
23 		 1.430888
24 		 1.320125
25 		 1.213627
26 		 1.112884
27 		 1.018870
28 		 0.932023
29 		 0.852306
30 		 0.779318
31 		 0.712431
32 		 0.650917
33 		 0.594058
34 		 0.541208
35 		 0.491839
36 		 0.445552
37 		 0.402069
38 		 0.361222
39 		 0.322926
40 		 0.287155
41 		 0.253918
42 		 0.223237
43 		 0.195127
44 		 0.169581
45 		 0.146566
46 		 0.126010
47 		 0.107811
48 		 0.091834
49 		 0.077922
50 		 0.065897
51 		 0.055577
52 		 0.046774
53 		 0.039305
54 		 0.032999
55 		 0.027695
56 		 0.023247
57 		 0.019525
58 		 0.016417
59 		 0.013823
60 		 0.011660
61 		 0.009854
62 		 0.008346
63 		 0.007085
64 		

In [6]:
print(np.dot(user_mat, movie_mat))
print(ratings_mat)

[[ 9.99999751  9.99999821  9.99999931 10.00000305]
 [10.00000026  4.00000024  9.00000011  9.99999967]
 [ 7.99999964  8.9999998   9.99999994  5.00000043]
 [ 8.99999996  8.00000001 10.00000003 10.00000006]
 [ 9.99999898  4.99999932  8.99999976  9.00000124]
 [ 6.00000059  4.00000042 10.00000019  5.99999932]
 [ 9.00000166  8.00000122 10.0000005   8.99999799]
 [10.00000074  5.00000051  9.0000002   7.99999913]
 [ 7.00000008  8.00000005 10.00000002  7.99999991]
 [ 8.99999865  4.99999904  8.99999963  7.00000164]
 [ 8.99999889  7.99999925  9.99999972  8.00000135]
 [ 9.00000035 10.00000026 10.00000013  8.99999961]
 [10.00000042  9.00000031 10.00000014  7.99999952]
 [ 5.00000001  8.00000006  5.00000003  7.99999997]
 [ 9.99999962  7.99999976  9.99999992 10.00000047]
 [ 9.00000051  9.00000039 10.00000017  9.99999938]
 [ 9.00000091  8.00000062  8.00000025  7.99999893]
 [10.0000002   8.00000013  1.00000005  9.99999977]
 [ 4.9999996   5.99999969  9.99999988 10.0000005 ]
 [ 8.00000036  7.00000022 10.00

**In this case, we were able to completely reconstruct the item-movie matrix to obtain an essentially 0 mean squared error. I obtained 0 MSE on iteration 165.**

The last time we placed an **np.nan** value into this matrix the entire svd algorithm in python broke.  Let's see if that is still the case using your FunkSVD function.  In the below cell, I have placed a nan into the first cell of your numpy array.  

`4.` Use 4 latent features, a learning rate of 0.005, and 250 iterations.  Are you able to run your SVD without it breaking (something that was not true about the python built in)?  Do you get a prediction for the nan value?  What is your prediction for the missing value? Use the cells below to answer these questions.

In [7]:
ratings_mat[0, 0] = np.nan
ratings_mat

matrix([[nan, 10., 10., 10.],
        [10.,  4.,  9., 10.],
        [ 8.,  9., 10.,  5.],
        [ 9.,  8., 10., 10.],
        [10.,  5.,  9.,  9.],
        [ 6.,  4., 10.,  6.],
        [ 9.,  8., 10.,  9.],
        [10.,  5.,  9.,  8.],
        [ 7.,  8., 10.,  8.],
        [ 9.,  5.,  9.,  7.],
        [ 9.,  8., 10.,  8.],
        [ 9., 10., 10.,  9.],
        [10.,  9., 10.,  8.],
        [ 5.,  8.,  5.,  8.],
        [10.,  8., 10., 10.],
        [ 9.,  9., 10., 10.],
        [ 9.,  8.,  8.,  8.],
        [10.,  8.,  1., 10.],
        [ 5.,  6., 10., 10.],
        [ 8.,  7., 10.,  7.]])

In [8]:
# run SVD on the matrix with the missing value
user_mat, movie_mat = FunkSVD(ratings_mat, latent_features=4, learning_rate=0.005, iters=250)

Optimizaiton Statistics
Iterations | Mean Squared Error 
1 		 47.354804
2 		 18.520434
3 		 5.088117
4 		 3.000466
5 		 2.743214
6 		 2.671300
7 		 2.628195
8 		 2.587504
9 		 2.542930
10 		 2.492590
11 		 2.435706
12 		 2.371932
13 		 2.301267
14 		 2.224090
15 		 2.141212
16 		 2.053882
17 		 1.963744
18 		 1.872727
19 		 1.782867
20 		 1.696099
21 		 1.614048
22 		 1.537867
23 		 1.468156
24 		 1.404970
25 		 1.347907
26 		 1.296247
27 		 1.249099
28 		 1.205534
29 		 1.164689
30 		 1.125827
31 		 1.088366
32 		 1.051880
33 		 1.016091
34 		 0.980839
35 		 0.946059
36 		 0.911748
37 		 0.877944
38 		 0.844698
39 		 0.812057
40 		 0.780048
41 		 0.748669
42 		 0.717886
43 		 0.687635
44 		 0.657822
45 		 0.628338
46 		 0.599066
47 		 0.569891
48 		 0.540716
49 		 0.511468
50 		 0.482109
51 		 0.452641
52 		 0.423110
53 		 0.393609
54 		 0.364268
55 		 0.335257
56 		 0.306772
57 		 0.279025
58 		 0.252232
59 		 0.226602
60 		 0.202324
61 		 0.179558
62 		 0.158424
63 		 0.139002
64 		

In [9]:
preds = np.dot(user_mat, movie_mat)
print("The predicted value for the missing rating is {}:".format(preds[0,0]))
print()
print("The actual value for the missing rating is {}:".format(ratings_mat[0,0]))
print()
assert np.isnan(preds[0,0]) == False
print("That's right! You just predicted a rating for a user-movie pair that was never rated!")
print("But if you look in the original matrix, this was actually a value of 10. Not bad!")

The predicted value for the missing rating is 10.437453690738774:

The actual value for the missing rating is nan:

That's right! You just predicted a rating for a user-movie pair that was never rated!
But if you look in the original matrix, this was actually a value of 10. Not bad!


Now let's extend this to a more realistic example. Unfortunately, running this function on your entire user-movie matrix is still not something you likely want to do on your local machine.  However, we can see how well this example extends to 1000 users.  In the above portion, you were using a very small subset of data with no missing values.

`5.` Given the size of this matrix, this will take quite a bit of time.  Consider the following hyperparameters: 4 latent features, 0.005 learning rate, and 20 iterations.  Grab a snack, take a walk, and this should be done running in a bit.

In [10]:
# Setting up a matrix of the first 1000 users with movie ratings
first_1000_users = np.matrix(user_by_movie.head(1000))

# perform funkSVD on the matrix of the top 1000 users
user_mat, movie_mat = FunkSVD(first_1000_users, latent_features=4, learning_rate=0.005, iters=20)

Optimizaiton Statistics
Iterations | Mean Squared Error 
1 		 23.441003
2 		 10.820521
3 		 7.430676
4 		 5.734211
5 		 4.662418
6 		 3.909402
7 		 3.346436
8 		 2.908852
9 		 2.559957
10 		 2.276766
11 		 2.043756
12 		 1.849893
13 		 1.687073
14 		 1.549212
15 		 1.431641
16 		 1.330710
17 		 1.243513
18 		 1.167702
19 		 1.101365
20 		 1.042942


`6.` Now that you have a set of predictions for each user-movie pair.  Let's answer a few questions about your results. Provide the correct values to each of the variables below, and check your solutions using the tests below.

In [13]:
# How many actual ratings exist in first_1000_users
num_ratings = np.count_nonzero(~np.isnan(first_1000_users))
print("The number of actual ratings in the first_1000_users is {}.".format(num_ratings))
print()

# How many ratings did we make for user-movie pairs that didn't have ratings
ratings_for_missing = first_1000_users.shape[0]*first_1000_users.shape[1] - num_ratings
print("The number of ratings made for user-movie pairs that didn't have ratings is {}".format(ratings_for_missing))

The number of actual ratings in the first_1000_users is 10852.

The number of ratings made for user-movie pairs that didn't have ratings is 31234148


In [14]:
# Test your results against the solution
assert num_ratings == 10852, "Oops!  The number of actual ratings doesn't quite look right."
assert ratings_for_missing == 31234148, "Oops!  The number of movie-user pairs that you made ratings for that didn't actually have ratings doesn't look right."

# Make sure you made predictions on all the missing user-movie pairs
preds = np.dot(user_mat, movie_mat)
assert np.isnan(preds).sum() == 0
print("Nice job!  Looks like you have predictions made for all the missing user-movie pairs! But I still have one question... How good are they?")

Nice job!  Looks like you have predictions made for all the missing user-movie pairs! But I still have one question... How good are they?
